# 2) Extensive form games

In the first tutorial, we used Gambit to set up the Prisoner's Dilemma, an example of a normal (strategic) form game.

Gambit can also be used to set up extensive form games; the game is represented as a tree, where each node represents a decision point for a player, and the branches represent the possible actions they can take.

**Example: One-shot trust game with binary actions**

[Kreps (1990)](#references) introduced a game commonly referred to as the **trust game**.
We will build a one-shot version of this game using Gambit's game transformation operations.

The game can be defined as follows:
- There are two players, a **Buyer** and a **Seller**.
- The Buyer moves first and has two actions, **Trust** or **Not trust**.
- If the Buyer chooses **Not trust**, then the game ends, and both players receive payoffs of `0`.
- If the Buyer chooses **Trust**, then the Seller has a choice with two actions, **Honor** or **Abuse**.
- If the Seller chooses **Honor**, both players receive payoffs of `1`;
- If the Seller chooses **Abuse**, the Buyer receives a payoff of `-1` and the Seller receives a payoff of `2`.

We create a game with an extensive representation using `Game.new_tree`:

In [ ]:
import pygambit as gbt

In [ ]:
g = gbt.Game.new_tree(
    players=["Buyer", "Seller"],
    title="One-shot trust game, after Kreps (1990)"
)

The tree of the game contains just a root node, with no children:

In [ ]:
len(g.root.children)

To extend a game from an existing terminal node, use `Game.append_move`. To begin with, the sole root node is the terminal node.

Here we extend the game from the root node by adding the first move for the "Buyer" player, creating two child nodes (one for each possible action).

In [ ]:
g.append_move(
    g.root,  # This is the node to append the move to
    player="Buyer",
    
    actions=["Trust", "Not trust"]
)
len(g.root.children)

We can also optionally specify labels for nodes when defining a game.
This isn't strictly necessary, but doing so makes the game easier to understand and work with than referring to nodes by their indices.

Here we'll label the nodes according to the actions that precede them in the game tree.

In [ ]:
for node in g.root.children:
    node.label = node.prior_action.label

We can then also add the Seller's move in the situation after the Buyer chooses Trust:

In [ ]:
g.append_move(
    g.root.children["Trust"],
    player="Seller",
    actions=["Honor", "Abuse"]
)
for node in g.root.children["Trust"].children:
    node.label = node.prior_action.label

Now that we have the moves of the game defined, we add payoffs.

Payoffs are associated with an `Outcome`; each `Outcome` has a vector of payoffs, one for each player, and optionally an identifying text label.

First we add the outcome associated with the Seller proving themselves trustworthy:

In [ ]:
g.set_outcome(
    g.root.children["Trust"].children["Honor"],
    outcome=g.add_outcome(
        payoffs=[1, 1],
        label="Trustworthy"
    )
)

Next, the outcome associated with the scenario where the Buyer trusts but the Seller does not return the trust:

In [ ]:
g.set_outcome(
    g.root.children["Trust"].children["Abuse"],
    outcome=g.add_outcome(
        payoffs=[-1, 2],
        label="Untrustworthy"
    )
)

And, finally the outcome associated with the Buyer opting out of the interaction:

In [ ]:
g.set_outcome(
    g.root.children["Not trust"],
    g.add_outcome(
        payoffs=[0, 0],
        label="Opt-out"
    )
)

Nodes without an outcome attached are assumed to have payoffs of zero for all players.

Therefore, adding the outcome to this latter terminal node is not strictly necessary in Gambit, but it is useful to be explicit for readability.

Saving and reading extensive form games to and from file
--------------------

You can use Gambit to save games to, and read from files.
The specific format depends on whether the game is normal or extensive form.

Here we'll save the Trust game (extensive form) to the `.efg` format.

In [ ]:
g.to_efg("games/trust_game.efg")

You can easily restore the game object from file like so:

In [ ]:
restored_game = gbt.read_efg("games/trust_game.efg")
type(restored_game)

Viewing the game with *draw_tree*
---------------------------------

The Gambit project has developed `draw_tree`, which is packaged separately from PyGambit and can be used to visualize extensive form games saved to `.efg` files.

In [ ]:
from draw_tree import draw_tree

In [ ]:
draw_tree("games/trust_game.efg")

#### References

Kreps, D. (1990) "Corporate Culture and Economic Theory." In J. Alt and K. Shepsle, eds., *Perspectives on Positive Political Economy*, Cambridge University Press.